In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
# Setting logging level
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

Loading the data of the [SLAC dataset](http://jmir.sourceforge.net/Codaich.html), with 250 MIDI files. The dataset is equally balanced over the classes.

The vectors are computed using [midi2vec](https://github.com/midi-ld/midi2vec).

The dataset provides 2 kind of classes (genres and subgenres), respectively of 5 and 10 elements:

- Blues
    - Modern Blues
    - Traditional Blues
- Classical
    - Baroque
    - Romantic
- Jazz
    - Bop
    - Swing
- Rap
    - Hardcore Rap
    - Pop Rap
- Rock
    - Alternative Rock
    - Metal

In [3]:
data = np.loadtxt('./dataset-slac/vectors.txt')
genres = np.loadtxt('./dataset-slac/genre.txt', dtype=np.str, delimiter=',')
subgenres = np.loadtxt('./dataset-slac/subgenre.txt', dtype=np.str, delimiter=',')

The labels are represented with the 1-hot encoding

In [4]:
encoder = LabelBinarizer()
def onehot_labels(labels):
    return encoder.fit_transform(labels)
    
def onehot2str(onehot):
    return encoder.inverse_transform(np.array([onehot]))[0]

### Experiment 1: Genres 

In [5]:
labels_1hot = onehot_labels(genres)
    
onehot2str(labels_1hot[0])

'Classical'

In [6]:
data.shape

(250, 100)

In [7]:
labels_1hot.shape

(250, 5)

In [8]:
num_input = data.shape[1]
num_output = labels_1hot.shape[1]

num_input, num_output

(100, 5)

We use a Neural Network with 3 dense layers, trained with the Adam Optimizer.

In [9]:
def create_model():
    model = tf.keras.Sequential()
    
    model.add(layers.Dense(100, activation='relu', input_shape=(num_input,)))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(num_output, activation='sigmoid'))

    model.compile(optimizer=tf.train.AdamOptimizer(0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [10]:
neural_network = KerasClassifier(build_fn=create_model, 
                                 epochs=100, 
                                 batch_size=32)

The network is evaluated with 10-fold cross-validation. The final score is the average of the results.

In [11]:
scores = cross_val_score(neural_network, data, labels_1hot, cv=10, verbose=0)
scores

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
225/225 [==============================] - 0s 709us/sample - loss: 0.4999 - acc: 0.7742
Epoch 2/100
225/225 [==============================] - 0s 60us/sample - loss: 0.2563 - acc: 0.9129
Epoch 3/100
225/225 [==============================] - 0s 66us/sample - loss: 0.1397 - acc: 0.9493
Epoch 4/100
225/225 [==============================] - 0s 60us/sample - loss: 0.0679 - acc: 0.9796
Epoch 5/100
225/225 [==============================] - 0s 75us/sample - loss: 0.0485 - acc: 0.9813
Epoch 6/100
225/225 [==============================] - 0s 62us/sample - loss: 0.0493 - acc: 0.9813
Epoch 7/100
225/225 [==============================] - 0s 70us/sample - loss: 0.0305 - acc: 0.9884
Epoch 8/100
225/225 [==============================] - 0s 75us/sample - loss: 0.0147 - acc: 0.9982
Epoch 9/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0081 - 

225/225 [==============================] - 0s 60us/sample - loss: 2.9618e-05 - acc: 1.0000
Epoch 75/100
225/225 [==============================] - 0s 61us/sample - loss: 2.8995e-05 - acc: 1.0000
Epoch 76/100
225/225 [==============================] - 0s 58us/sample - loss: 2.8300e-05 - acc: 1.0000
Epoch 77/100
225/225 [==============================] - 0s 49us/sample - loss: 2.7520e-05 - acc: 1.0000
Epoch 78/100
225/225 [==============================] - 0s 52us/sample - loss: 2.6697e-05 - acc: 1.0000
Epoch 79/100
225/225 [==============================] - 0s 54us/sample - loss: 2.6061e-05 - acc: 1.0000
Epoch 80/100
225/225 [==============================] - 0s 57us/sample - loss: 2.5339e-05 - acc: 1.0000
Epoch 81/100
225/225 [==============================] - 0s 61us/sample - loss: 2.4744e-05 - acc: 1.0000
Epoch 82/100
225/225 [==============================] - 0s 58us/sample - loss: 2.4075e-05 - acc: 1.0000
Epoch 83/100
225/225 [==============================] - 0s 58us/sample - loss

225/225 [==============================] - 0s 52us/sample - loss: 4.3998e-05 - acc: 1.0000
Epoch 53/100
225/225 [==============================] - 0s 52us/sample - loss: 4.2154e-05 - acc: 1.0000
Epoch 54/100
225/225 [==============================] - 0s 48us/sample - loss: 4.0719e-05 - acc: 1.0000
Epoch 55/100
225/225 [==============================] - 0s 60us/sample - loss: 4.1135e-05 - acc: 1.0000
Epoch 56/100
225/225 [==============================] - 0s 53us/sample - loss: 4.3667e-05 - acc: 1.0000
Epoch 57/100
225/225 [==============================] - 0s 54us/sample - loss: 4.1242e-05 - acc: 1.0000
Epoch 58/100
225/225 [==============================] - 0s 52us/sample - loss: 3.6938e-05 - acc: 1.0000
Epoch 59/100
225/225 [==============================] - 0s 50us/sample - loss: 3.5402e-05 - acc: 1.0000
Epoch 60/100
225/225 [==============================] - 0s 51us/sample - loss: 3.3096e-05 - acc: 1.0000
Epoch 61/100
225/225 [==============================] - 0s 56us/sample - loss

225/225 [==============================] - 0s 63us/sample - loss: 1.4751e-04 - acc: 1.0000
Epoch 31/100
225/225 [==============================] - 0s 61us/sample - loss: 1.3595e-04 - acc: 1.0000
Epoch 32/100
225/225 [==============================] - 0s 55us/sample - loss: 1.2808e-04 - acc: 1.0000
Epoch 33/100
225/225 [==============================] - 0s 60us/sample - loss: 1.2086e-04 - acc: 1.0000
Epoch 34/100
225/225 [==============================] - 0s 55us/sample - loss: 1.1285e-04 - acc: 1.0000
Epoch 35/100
225/225 [==============================] - 0s 58us/sample - loss: 1.0633e-04 - acc: 1.0000
Epoch 36/100
225/225 [==============================] - 0s 58us/sample - loss: 1.0022e-04 - acc: 1.0000
Epoch 37/100
225/225 [==============================] - 0s 61us/sample - loss: 9.5522e-05 - acc: 1.0000
Epoch 38/100
225/225 [==============================] - 0s 59us/sample - loss: 9.0494e-05 - acc: 1.0000
Epoch 39/100
225/225 [==============================] - 0s 62us/sample - loss

225/225 [==============================] - 0s 57us/sample - loss: 0.0052 - acc: 0.9991
Epoch 9/100
225/225 [==============================] - 0s 55us/sample - loss: 0.0038 - acc: 0.9991
Epoch 10/100
225/225 [==============================] - 0s 57us/sample - loss: 0.0027 - acc: 0.9991
Epoch 11/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0016 - acc: 1.0000
Epoch 12/100
225/225 [==============================] - 0s 57us/sample - loss: 0.0012 - acc: 1.0000
Epoch 13/100
225/225 [==============================] - 0s 58us/sample - loss: 9.0012e-04 - acc: 1.0000
Epoch 14/100
225/225 [==============================] - 0s 56us/sample - loss: 6.9147e-04 - acc: 1.0000
Epoch 15/100
225/225 [==============================] - 0s 55us/sample - loss: 5.1182e-04 - acc: 1.0000
Epoch 16/100
225/225 [==============================] - 0s 58us/sample - loss: 4.4414e-04 - acc: 1.0000
Epoch 17/100
225/225 [==============================] - 0s 55us/sample - loss: 3.7573e-04 - acc: 1

...

Epoch 100/100
25/25 [==============================] - 0s 4ms/sample - loss: 0.5077 - acc: 0.9200
Epoch 1/100
225/225 [==============================] - 0s 936us/sample - loss: 0.5042 - acc: 0.7502
Epoch 2/100
225/225 [==============================] - 0s 66us/sample - loss: 0.2300 - acc: 0.9209
Epoch 3/100
225/225 [==============================] - 0s 70us/sample - loss: 0.1125 - acc: 0.9627
Epoch 4/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0639 - acc: 0.9769
Epoch 5/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0385 - acc: 0.9867
Epoch 6/100
225/225 [==============================] - 0s 69us/sample - loss: 0.0188 - acc: 0.9947
Epoch 7/100
225/225 [==============================] - 0s 66us/sample - loss: 0.0110 - acc: 0.9982
Epoch 8/100
225/225 [==============================] - 0s 65us/sample - loss: 0.0098 - acc: 0.9973
Epoch 9/100
225/225 [==============================] - 0s 64us/sample - loss: 0.0049 - acc: 0.9991
Epoch 10/1

225/225 [==============================] - 0s 65us/sample - loss: 1.7873e-05 - acc: 1.0000
Epoch 79/100
225/225 [==============================] - 0s 65us/sample - loss: 1.7396e-05 - acc: 1.0000
Epoch 80/100
225/225 [==============================] - 0s 58us/sample - loss: 1.7056e-05 - acc: 1.0000
Epoch 81/100
225/225 [==============================] - 0s 64us/sample - loss: 1.6592e-05 - acc: 1.0000
Epoch 82/100
225/225 [==============================] - 0s 64us/sample - loss: 1.6472e-05 - acc: 1.0000
Epoch 83/100
225/225 [==============================] - 0s 61us/sample - loss: 1.6228e-05 - acc: 1.0000
Epoch 84/100
225/225 [==============================] - 0s 64us/sample - loss: 1.5916e-05 - acc: 1.0000
Epoch 85/100
225/225 [==============================] - 0s 65us/sample - loss: 1.5552e-05 - acc: 1.0000
Epoch 86/100
225/225 [==============================] - 0s 62us/sample - loss: 1.5150e-05 - acc: 1.0000
Epoch 87/100
225/225 [==============================] - 0s 64us/sample - loss

225/225 [==============================] - 0s 63us/sample - loss: 3.9055e-05 - acc: 1.0000
Epoch 57/100
225/225 [==============================] - 0s 60us/sample - loss: 3.7965e-05 - acc: 1.0000
Epoch 58/100
225/225 [==============================] - 0s 72us/sample - loss: 3.6795e-05 - acc: 1.0000
Epoch 59/100
225/225 [==============================] - 0s 67us/sample - loss: 3.5618e-05 - acc: 1.0000
Epoch 60/100
225/225 [==============================] - 0s 68us/sample - loss: 3.4483e-05 - acc: 1.0000
Epoch 61/100
225/225 [==============================] - 0s 68us/sample - loss: 3.3406e-05 - acc: 1.0000
Epoch 62/100
225/225 [==============================] - 0s 67us/sample - loss: 3.2376e-05 - acc: 1.0000
Epoch 63/100
225/225 [==============================] - 0s 63us/sample - loss: 3.1371e-05 - acc: 1.0000
Epoch 64/100
225/225 [==============================] - 0s 65us/sample - loss: 3.0416e-05 - acc: 1.0000
Epoch 65/100
225/225 [==============================] - 0s 63us/sample - loss

225/225 [==============================] - 0s 65us/sample - loss: 4.0255e-05 - acc: 1.0000
Epoch 35/100
225/225 [==============================] - 0s 78us/sample - loss: 3.8754e-05 - acc: 1.0000
Epoch 36/100
225/225 [==============================] - 0s 57us/sample - loss: 3.5442e-05 - acc: 1.0000
Epoch 37/100
225/225 [==============================] - 0s 78us/sample - loss: 3.3511e-05 - acc: 1.0000
Epoch 38/100
225/225 [==============================] - 0s 65us/sample - loss: 3.1780e-05 - acc: 1.0000
Epoch 39/100
225/225 [==============================] - 0s 65us/sample - loss: 3.0038e-05 - acc: 1.0000
Epoch 40/100
225/225 [==============================] - 0s 69us/sample - loss: 2.8481e-05 - acc: 1.0000
Epoch 41/100
225/225 [==============================] - 0s 65us/sample - loss: 2.7116e-05 - acc: 1.0000
Epoch 42/100
225/225 [==============================] - 0s 69us/sample - loss: 2.5921e-05 - acc: 1.0000
Epoch 43/100
225/225 [==============================] - 0s 64us/sample - loss

array([0.92000002, 0.86400002, 0.88      , 0.88800001, 0.87199998,
       0.93599999, 0.9199999 , 0.91199994, 1.        , 0.95999998])

In [12]:
np.mean(scores)

0.9151999831199646

### Experiment 2: Subgenres 

The experiment is the same, but applied to the 10 subgenre classes.

In [13]:
labels_1hot = onehot_labels(subgenres)
    
onehot2str(labels_1hot[0])

'Romantic'

In [14]:
num_output = labels_1hot.shape[1]

num_input, num_output

(100, 10)

In [15]:
neural_network = KerasClassifier(build_fn=create_model, 
                                 epochs=100, 
                                 batch_size=32)

In [16]:
scores = cross_val_score(neural_network, data, labels_1hot, cv=10, verbose=0)
scores

Epoch 1/100
225/225 [==============================] - 0s 991us/sample - loss: 0.4086 - acc: 0.8467
Epoch 2/100
225/225 [==============================] - 0s 83us/sample - loss: 0.2600 - acc: 0.9067
Epoch 3/100
225/225 [==============================] - 0s 85us/sample - loss: 0.1728 - acc: 0.9276
Epoch 4/100
225/225 [==============================] - 0s 70us/sample - loss: 0.1203 - acc: 0.9569
Epoch 5/100
225/225 [==============================] - 0s 76us/sample - loss: 0.0849 - acc: 0.9671
Epoch 6/100
225/225 [==============================] - 0s 82us/sample - loss: 0.0635 - acc: 0.9773
Epoch 7/100
225/225 [==============================] - 0s 66us/sample - loss: 0.0529 - acc: 0.9796
Epoch 8/100
225/225 [==============================] - 0s 83us/sample - loss: 0.0407 - acc: 0.9871
Epoch 9/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0473 - acc: 0.9836
Epoch 10/100
225/225 [==============================] - 0s 70us/sample - loss: 0.0295 - acc: 0.9911
Epoch 11

225/225 [==============================] - 0s 61us/sample - loss: 0.0145 - acc: 0.9991
Epoch 84/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0145 - acc: 0.9991
Epoch 85/100
225/225 [==============================] - 0s 75us/sample - loss: 0.0145 - acc: 0.9991
Epoch 86/100
225/225 [==============================] - 0s 75us/sample - loss: 0.0145 - acc: 0.9991
Epoch 87/100
225/225 [==============================] - 0s 75us/sample - loss: 0.0145 - acc: 0.9991
Epoch 88/100
225/225 [==============================] - 0s 70us/sample - loss: 0.0145 - acc: 0.9991
Epoch 89/100
225/225 [==============================] - 0s 65us/sample - loss: 0.0144 - acc: 0.9991
Epoch 90/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0144 - acc: 0.9991
Epoch 91/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0144 - acc: 0.9991
Epoch 92/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0144 - acc: 0.9991
Epoch 93/100


225/225 [==============================] - 0s 66us/sample - loss: 2.1466e-04 - acc: 1.0000
Epoch 64/100
225/225 [==============================] - 0s 67us/sample - loss: 2.0856e-04 - acc: 1.0000
Epoch 65/100
225/225 [==============================] - 0s 71us/sample - loss: 1.9857e-04 - acc: 1.0000
Epoch 66/100
225/225 [==============================] - 0s 62us/sample - loss: 1.9353e-04 - acc: 1.0000
Epoch 67/100
225/225 [==============================] - 0s 65us/sample - loss: 1.8522e-04 - acc: 1.0000
Epoch 68/100
225/225 [==============================] - 0s 64us/sample - loss: 1.9351e-04 - acc: 1.0000
Epoch 69/100
225/225 [==============================] - 0s 65us/sample - loss: 1.6815e-04 - acc: 1.0000
Epoch 70/100
225/225 [==============================] - 0s 63us/sample - loss: 2.0293e-04 - acc: 1.0000
Epoch 71/100
225/225 [==============================] - 0s 64us/sample - loss: 2.1041e-04 - acc: 1.0000
Epoch 72/100
225/225 [==============================] - 0s 65us/sample - loss

225/225 [==============================] - 0s 66us/sample - loss: 2.5960e-04 - acc: 1.0000
Epoch 43/100
225/225 [==============================] - 0s 61us/sample - loss: 0.0028 - acc: 0.9991
Epoch 44/100
225/225 [==============================] - 0s 63us/sample - loss: 0.0031 - acc: 0.9991
Epoch 45/100
225/225 [==============================] - 0s 64us/sample - loss: 4.2126e-04 - acc: 1.0000
Epoch 46/100
225/225 [==============================] - 0s 62us/sample - loss: 8.0238e-04 - acc: 1.0000
Epoch 47/100
225/225 [==============================] - 0s 65us/sample - loss: 7.9714e-04 - acc: 0.9996
Epoch 48/100
225/225 [==============================] - 0s 63us/sample - loss: 9.7536e-04 - acc: 0.9996
Epoch 49/100
225/225 [==============================] - 0s 65us/sample - loss: 7.1862e-04 - acc: 1.0000
Epoch 50/100
225/225 [==============================] - 0s 67us/sample - loss: 5.5459e-04 - acc: 1.0000
Epoch 51/100
225/225 [==============================] - 0s 64us/sample - loss: 4.2106

225/225 [==============================] - 0s 67us/sample - loss: 9.3890e-04 - acc: 1.0000
Epoch 22/100
225/225 [==============================] - 0s 69us/sample - loss: 8.1547e-04 - acc: 1.0000
Epoch 23/100
225/225 [==============================] - 0s 81us/sample - loss: 7.6406e-04 - acc: 1.0000
Epoch 24/100
225/225 [==============================] - 0s 70us/sample - loss: 6.2168e-04 - acc: 1.0000
Epoch 25/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0043 - acc: 0.9987
Epoch 26/100
225/225 [==============================] - 0s 66us/sample - loss: 0.0043 - acc: 0.9982
Epoch 27/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0050 - acc: 0.9987
Epoch 28/100
225/225 [==============================] - 0s 67us/sample - loss: 0.0054 - acc: 0.9982
Epoch 29/100
225/225 [==============================] - 0s 66us/sample - loss: 0.0038 - acc: 0.9991
Epoch 30/100
225/225 [==============================] - 0s 63us/sample - loss: 0.0051 - acc: 0.99

...

225/225 [==============================] - 0s 66us/sample - loss: 0.0041 - acc: 0.9991
Epoch 17/100
225/225 [==============================] - 0s 68us/sample - loss: 0.0035 - acc: 0.9987
Epoch 18/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0031 - acc: 0.9996
Epoch 19/100
225/225 [==============================] - 0s 69us/sample - loss: 0.0028 - acc: 0.9991
Epoch 20/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0028 - acc: 0.9991
Epoch 21/100
225/225 [==============================] - 0s 68us/sample - loss: 0.0030 - acc: 0.9991
Epoch 22/100
225/225 [==============================] - 0s 71us/sample - loss: 0.0043 - acc: 0.9982
Epoch 23/100
225/225 [==============================] - 0s 72us/sample - loss: 0.0026 - acc: 0.9991
Epoch 24/100
225/225 [==============================] - 0s 66us/sample - loss: 0.0020 - acc: 0.9991
Epoch 25/100
225/225 [==============================] - 0s 70us/sample - loss: 0.0014 - acc: 0.9996
Epoch 26/100


225/225 [==============================] - 0s 64us/sample - loss: 8.2013e-05 - acc: 1.0000
Epoch 96/100
225/225 [==============================] - 0s 72us/sample - loss: 6.4099e-05 - acc: 1.0000
Epoch 97/100
225/225 [==============================] - 0s 70us/sample - loss: 7.2512e-05 - acc: 1.0000
Epoch 98/100
225/225 [==============================] - 0s 69us/sample - loss: 6.6091e-05 - acc: 1.0000
Epoch 99/100
225/225 [==============================] - 0s 70us/sample - loss: 6.1488e-05 - acc: 1.0000
Epoch 100/100
25/25 [==============================] - 0s 7ms/sample - loss: 0.6462 - acc: 0.9000
Epoch 1/100
225/225 [==============================] - 0s 1ms/sample - loss: 0.4011 - acc: 0.8507
Epoch 2/100
225/225 [==============================] - 0s 70us/sample - loss: 0.2474 - acc: 0.9124
Epoch 3/100
225/225 [==============================] - 0s 71us/sample - loss: 0.1623 - acc: 0.9298
Epoch 4/100
225/225 [==============================] - 0s 70us/sample - loss: 0.1114 - acc: 0.9542


225/225 [==============================] - 0s 74us/sample - loss: 1.7220e-04 - acc: 1.0000
Epoch 75/100
225/225 [==============================] - 0s 77us/sample - loss: 1.7951e-04 - acc: 1.0000
Epoch 76/100
225/225 [==============================] - 0s 74us/sample - loss: 1.3708e-04 - acc: 1.0000
Epoch 77/100
225/225 [==============================] - 0s 74us/sample - loss: 1.6251e-04 - acc: 1.0000
Epoch 78/100
225/225 [==============================] - 0s 79us/sample - loss: 1.2187e-04 - acc: 1.0000
Epoch 79/100
225/225 [==============================] - 0s 66us/sample - loss: 1.4122e-04 - acc: 1.0000
Epoch 80/100
225/225 [==============================] - 0s 68us/sample - loss: 1.2033e-04 - acc: 1.0000
Epoch 81/100
225/225 [==============================] - 0s 77us/sample - loss: 1.0765e-04 - acc: 1.0000
Epoch 82/100
225/225 [==============================] - 0s 72us/sample - loss: 9.8497e-05 - acc: 1.0000
Epoch 83/100
225/225 [==============================] - 0s 75us/sample - loss

225/225 [==============================] - 0s 66us/sample - loss: 1.7245e-04 - acc: 1.0000
Epoch 54/100
225/225 [==============================] - 0s 68us/sample - loss: 2.1373e-04 - acc: 1.0000
Epoch 55/100
225/225 [==============================] - 0s 67us/sample - loss: 1.2282e-04 - acc: 1.0000
Epoch 56/100
225/225 [==============================] - 0s 69us/sample - loss: 2.0505e-04 - acc: 1.0000
Epoch 57/100
225/225 [==============================] - 0s 70us/sample - loss: 1.7762e-04 - acc: 1.0000
Epoch 58/100
225/225 [==============================] - 0s 75us/sample - loss: 1.2479e-04 - acc: 1.0000
Epoch 59/100
225/225 [==============================] - 0s 73us/sample - loss: 1.3122e-04 - acc: 1.0000
Epoch 60/100
225/225 [==============================] - 0s 65us/sample - loss: 1.1412e-04 - acc: 1.0000
Epoch 61/100
225/225 [==============================] - 0s 67us/sample - loss: 1.1078e-04 - acc: 1.0000
Epoch 62/100
225/225 [==============================] - 0s 63us/sample - loss

array([0.9199999 , 0.92799997, 0.89599997, 0.89999992, 0.94399995,
       0.90399992, 0.89599997, 0.89999998, 0.94399995, 0.94      ])

In [17]:
np.mean(scores)

0.9171999514102935